In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
%store -r df

In [ ]:
df.drop(columns=['cert_date', 'archived_date', 'manufacturer', 'protection_profiles', 'defenses', 'crypto_algs'], inplace=True)
#df.drop(columns=['category', 'scheme'], inplace=True) # Uncomment if using PCA

In [ ]:
df.drop(columns=['cert_lab', 'n_pages'], inplace=True)

In [ ]:
features = df.loc[:,['sec_level_cat', 'n_updates', 'pdf_encrypted', 'n_defenses', 'n_crypto_algs', 'n_protection_profiles', 'n_pages_cat', 'n_updates_cat', 'cert_year', 'scheme_cat']]
labels = df.sec_level_cat.astype('int')

In [ ]:
features.pdf_encrypted = features.pdf_encrypted.map(lambda x: 1 if x is True else 0)
dummies = pd.get_dummies(df.scheme_cat, drop_first=True)
features = pd.concat([features, dummies], axis=1)
features.drop(columns=['scheme_cat'], inplace=True)

In [ ]:
features.head()

In [ ]:
features.sec_level_cat = features.sec_level_cat.astype('int')
features.sec_level_cat.value_counts()

In [ ]:
subset = features.loc[(features.sec_level_cat == 1) | (features.sec_level_cat == 10),:]
#subset.category = subset.category.astype('category').cat.codes

In [ ]:
subset.head()

## Show interesting scatter plot

In [ ]:
fig, ax = plt.subplots()
subset.plot(kind='scatter', x='n_pages_cat', y='n_defenses', s=50, c='sec_level_cat', cmap='viridis', ax=ax, figsize=(10,7))

In [ ]:
# features = subset.drop(columns=['sec_level_cat'])
# labels = subset.loc[:,['sec_level_cat']]

labels = features.loc[:,['sec_level_cat']]
features = features.drop(columns=['sec_level_cat'])

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, random_state=42, stratify=labels)

In [ ]:
features.head()

In [ ]:
stdsc = StandardScaler()
x_train_std = stdsc.fit_transform(x_train)
x_test_std = stdsc.transform(x_test)

In [ ]:
working_dir = '/Users/adam/phd/projects/certificates/data_dir/'
tf_data = tf.Variable(features)
labels = labels.tolist()
np.savetxt(os.path.join(working_dir, 'metadata.tsv'), labels, delimiter='\t')

In [ ]:
with tf.Session() as sess:
    saver = tf.train.Saver([tf_data])
    sess.run(tf_data.initializer)
    saver.save(sess, os.path.join(working_dir, 'tf_data.cpkt'))
    config = projector.ProjectorConfig()
    
    embedding = config.embeddings.add()
    embedding.tensor_name = tf_data.name

    # Link this tensor to its metadata(Labels) file
    embedding.metadata_path = 'metadata.tsv'
        
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(working_dir), config)


In [ ]:
def plot_decision_regions(X, y, classifier, resolution=0.02):
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.ylim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0],
                    y=X[y == cl, 1],
                    alpha=0.6,
                    c=cmap(idx),
                    edgecolor='black',
                    marker=markers[idx],
                    label=cl)

In [ ]:
pca = PCA(n_components=2)
lr = LogisticRegression()
x_train_pca = pca.fit_transform(x_train_std)
x_test_pca = pca.transform(x_test_std)
lr.fit(x_train_pca, y_train.to_numpy().ravel())
plot_decision_regions(x_train_pca, y_train.to_numpy().ravel(), lr)

In [ ]:
%store x_train_std
%store y_train
%store x_test_std
%store y_test

In [ ]:
y_train.nunique()

In [ ]:
x_train_std.shape